In [1]:
import gym
import tianshou as ts
import torch

In [2]:
envName = "CartPole-v1"
env = gym.make(envName)

In [3]:
train_envs = ts.env.DummyVectorEnv([lambda: gym.make(envName) for _ in range(10)])
test_envs = ts.env.DummyVectorEnv([lambda: gym.make(envName, render_mode="human") for _ in range(100)])

In [4]:
from tianshou.utils.net.common import Net
state_shape = env.observation_space.shape or env.observation_space.n
action_shape = env.action_space.shape or env.action_space.n
net = Net(state_shape, action_shape, hidden_sizes=(128,128,128,128))
optim = torch.optim.Adam(net.parameters(), lr=1e-3)

In [5]:
policy = ts.policy.DQNPolicy(net, optim, discount_factor=0.9, estimation_step=3, target_update_freq=320)

In [6]:
train_collector = ts.data.Collector(policy, train_envs, ts.data.VectorReplayBuffer(20000, 10), exploration_noise=True)
test_collector = ts.data.Collector(policy, train_envs, exploration_noise=True)

In [7]:
env.spec

EnvSpec(id='CartPole-v1', entry_point='gym.envs.classic_control.cartpole:CartPoleEnv', reward_threshold=475.0, nondeterministic=False, max_episode_steps=500, order_enforce=True, autoreset=False, disable_env_checker=False, apply_api_compatibility=False, kwargs={}, namespace=None, name='CartPole', version=1)

In [8]:
env.spec.reward_threshold

475.0

In [9]:
result = ts.trainer.offpolicy_trainer(
    policy, train_collector, test_collector,
    max_epoch=20, step_per_epoch=10000, step_per_collect=10,
    update_per_step=0.1, episode_per_test=100, batch_size=64,
    train_fn=lambda epoch, env_step: policy.set_eps(0.1),
    test_fn=lambda epoch, env_step: policy.set_eps(0.05),
    stop_fn=lambda mean_rewards: mean_rewards >= env.spec.reward_threshold,
)
print(f"Finished training! Use {result['duration']}")

Epoch #1: 10001it [00:08, 1134.79it/s, env_step=10000, len=198, loss=0.299, n/ep=0, n/st=10, rew=198.00]                           


Epoch #1: test_reward: 210.260000 ± 56.162197, best_reward: 210.260000 ± 56.162197 in #1


Epoch #2:   3%|3         | 330/10000 [00:00<00:07, 1269.23it/s, env_step=10320, len=198, loss=0.348, n/ep=0, n/st=10, rew=198.00]


KeyboardInterrupt: 

In [ ]:
result

{'duration': '186.48s',
 'train_time/model': '86.97s',
 'test_step': 857772,
 'test_episode': 5400,
 'test_time': '77.86s',
 'test_speed': '11017.17 step/s',
 'best_reward': 500.0,
 'best_result': '500.00 ± 0.00',
 'train_step': 137170,
 'train_episode': 932,
 'train_time/collector': '21.65s',
 'train_speed': '1262.85 step/s'}

In [ ]:
policy.eval()
policy.set_eps(0.05)
collector = ts.data.Collector(policy, gym.make(envName, render_mode="human"), exploration_noise=False)
collector.collect(n_episode=10, render=1 / 60)

c:\Users\TeamD\.conda\envs\tianshou\lib\site-packages\tianshou\data\collector.py:68: UserWarning: Single environment detected, wrap to DummyVectorEnv.
  warnings.warn("Single environment detected, wrap to DummyVectorEnv.")


KeyboardInterrupt: 